# Overview

I define a de Neural Network in notebook [Build the Neural Network](https://www.kaggle.com/code/aisuko/build-the-neural-network). Let's define a new neural network with a process built into PyTorch called convolution to recognize images. Convolution adds each element of an images to its local neighbors, weighted by a kernel, or a small matrix, that helps us extract certain features(like **edge detection, sharpness, blurriness**, etc.) from the input image.

Using convolution, we will define our model to take 1 input image channel, and output match our target of 10 labels representing numbers 0 through 9. And we will follow a standard MNIST algorithm.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        
        # first 2D convolutional layer, taking in 1 input channel (image),
        # outputting 32 convolutional features, with a square kernel size of 3
        self.conv1=nn.Conv2d(1,32,3,1)
        
        # second 2D convolutional layer, taking in the 32 input layers,
        # outputting 64 convolutional features, with a square kernel size of 3
        self.conv2=nn.Conv2d(32,64,3,1)
        
        # designed to ensure that adjacent pixels are either all 0s or all active with an input probability
        self.dropout1=nn.Dropout2d(0.25)
        self.dropout2=nn.Dropout2d(0.5)
        
        # first fully connected layer
        self.fc1=nn.Linear(9216, 128)
        # second fully connected layer that outputs our 10 labels
        self.fc2=nn.Linear(128,10)
    
    # x represents our data, we pass the data into the computation graph(neural network)
    # This will represent our feed-forward algorithm
    def forward(self, x):
        # pass data through conv1
        x=self.conv1(x)
        # use the rectified-linear activation function over x
        x=F.relu(x)
        
        x=self.conv2(x)
        x=F.relu(x)
        
        # run max pooling over x
        x=F.max_pool2d(x,2)
        # pass data through dropout1
        x=self.dropout1(x)
        # flattern x with start_dim=1
        x=torch.flatten(x,1)
        # pass data through 'fc1'
        x=self.fc1(x)
        x=F.relu(x)
        x=self.dropout2(x)
        x=self.fc2(x)
        
        # apply softmax to x
        output=F.log_softmax(x, dim=1)
        
        return output
        
        
my_nn=NeuralNetwork() # .to(device) supports to moving the model to a GPU
my_nn

NeuralNetwork(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (dropout1): Dropout2d(p=0.25, inplace=False)
  (dropout2): Dropout2d(p=0.5, inplace=False)
  (fc1): Linear(in_features=9216, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)

# Pass data through the model to test

In [2]:
image_in_tensor=torch.rand((1,1,28,28))
result=my_nn(image_in_tensor)
result

/opt/conda/lib/python3.10/site-packages/torch/nn/functional.py:1345: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


tensor([[-2.2642, -2.2925, -2.2536, -2.3206, -2.3308, -2.2335, -2.1965, -2.4381,
         -2.3717, -2.3472]], grad_fn=<LogSoftmaxBackward0>)

Each number in this resulting tensor equates to the prediction of the label the random tensor is associated to.